In [16]:
from src.models.predict_model import OHLProductionModel
import torch
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [86]:
model = OHLProductionModel(n_classes=296, device='cpu')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'pre_clas

In [4]:
checkpoint_path = '../models/chkpts/epoch=7-val_f1=0.96-val_accuracy=0.99-val_loss=0.0215.ckpt'
state_dict = torch.load(checkpoint_path, map_location=torch.device(model.device))[
    "state_dict"
]
model.load_state_dict(state_dict)
model.eval().to(model.device)
model.warmup()


/home/andrey/roonyx_projects/question-matching/src/models/predict_model.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x.logits)


In [8]:
i = model.preprocess_input('blood count')
i = {k: v.unsqueeze(0) for k, v in i.items()}
o = model(i).cpu().detach().numpy()

/home/andrey/roonyx_projects/question-matching/src/models/predict_model.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x.logits)


In [15]:
df = pd.read_csv('../data/raw/NLP training data 0816.csv')
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, )
df.head()

,index,question_id,client_id,campaign_id,question_category,question_subcategory,question_keyword,question_description
0,0,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,Which plan(s) cover preventive benefits?
1,1,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Preventive care,How much does preventive benefits cost to me?
2,2,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,Which plan(s) cover preventive benefits?
3,3,benefit_search-preventive_care-2,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Abdominal aortic aneurysm screening,How much does preventive benefits cost to me?
4,4,benefit_search-preventive_care-1,Aetna_Medicare,PDP_Conversion_SMS,Benefit Search,preventive care,Alcohol misuse screenings & counseling,Which plan(s) cover preventive benefits?


In [17]:
l_encoder = LabelEncoder()
l_encoder = l_encoder.fit(df["question_id"].values)

In [84]:
%%time
i = model.preprocess_input('Which plan(s) cover my drug(s)')
i = {k: v.unsqueeze(0) for k, v in i.items()}
o = model(i).cpu().detach().numpy()
idx = o.argsort(axis=1)


preds = []
for e in idx[0,-5:][::-1]:
    temp_dict = {}
    temp_dict['class_name'] = l_encoder.classes_[e]
    temp_dict['p'] = o[0,e]
    temp_dict['q'] = df[df.question_id==temp_dict['class_name']]['question_description'].values[0]
    preds.append(temp_dict)
    # print(f"{class_name}: {p:%} q: {q}")
    
preds_df = pd.DataFrame(preds)
preds_df.head()

CPU times: user 666 ms, sys: 7.74 ms, total: 674 ms
Wall time: 113 ms


/home/andrey/roonyx_projects/question-matching/src/models/predict_model.py:35: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x.logits)


,class_name,p,q
0,benefit_search-chemotherapy_drugs-1,0.286586,Which plan(s) cover chemotherapy drugs?
1,benefit_term-creditable_prescription_drug_cove...,0.125813,What does creditable prescription drug coverag...
2,benefit_search-inpatient_alcohol/drug_use_or_i...,0.090937,Which plan(s) cover inpatient treatment for al...
3,benefit_search-OTC_drugs-1,0.084257,Which plan(s) cover over-the-counter (OTC)?
4,benefit_term-generic_drug,0.052316,What does generic drug mean?
